In [53]:
import pandas as pd
ground_truth = pd.read_json('groud_truth_embedding.json')
targetStr = 'output/finetune_model_responses_qwen.csv'
targetDf = pd.read_csv(targetStr)

In [54]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel
import numpy as np
from tqdm import tqdm  # For progress bar
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')
model.eval()  # Set to evaluation mode

def get_embedding(text):
    # Tokenize and process the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    
    with torch.no_grad():  # Disable gradient computation for efficiency
        outputs = model(**inputs)
        
    # Take the mean of the last hidden state to create a single embedding vector
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embeddings

embeddings = []
for text in tqdm(targetDf['Generated Response'], desc="Generating embeddings"):
    embeddings.append(get_embedding(text))
targetDf['Embeddings_Generated'] = embeddings

Generating embeddings: 100%|██████████| 437/437 [00:44<00:00,  9.84it/s]


In [55]:
misconception_df = pd.read_csv('../embedding_generator/misconception_mapping.csv')
from sklearn.metrics.pairwise import cosine_similarity
def find_most_similar_id(target_embedding, ground_truth_df):
    # Calculate cosine similarity between the target embedding and each embedding in ground_truth
    similarities = cosine_similarity([target_embedding], list(ground_truth_df['Embedding']))
    
    # Find the index of the highest similarity score
    most_similar_index = np.argmax(similarities)
    
    # Retrieve the misconception_id with the highest similarity
    most_similar_id = ground_truth_df.iloc[most_similar_index]['MisconceptionId']
    return most_similar_id

# Apply the function to each row in targetDf
targetDf['prediction_result'] = targetDf['Embeddings_Generated'].apply(
    lambda emb: find_most_similar_id(emb, ground_truth)
)

In [56]:
targetDf.rename(columns={'prediction_result': 'MisconceptionId'}, inplace=True)

targetDf = targetDf.merge(misconception_df, on='MisconceptionId', how='left')

In [57]:
print(sum(targetDf['Expected Misconception'] == targetDf['MisconceptionName']) / len(targetDf))

0.0


In [58]:
print(targetDf['Generated Response'][100])

# of a line to the following equation by adding the same number to both sides: 2x + 3 = 7
to the same number on both sides of an equation. The given equation is \(2x + 3 = 7\). To solve for \(x\), we need to isolate \(x\) on one side. 

1. Subtract 3 from both sides of the equation:
   \[
   2x + 3 - 3 = 7 - 3
   \]
   Simplifying this gives:
   \[
   2x = 4
   \]

2. Divide both sides by 2:
   \[
   \frac{2x}{2} = \frac{4}{2}
   \]
   Simplifying this gives:
   \[
   x = 2
   \]

So, the value of \(x\) that satisfies the equation is \(x = 2\). Let's check it:

Substitute \(x = 2\) back into the original equation:
\[
2(2) + 3 = 4 + 3 = 7
\]

The solution checks out. Therefore, the final answer is:
\[
\boxed{2}
\] To solve the equation \(2x + 3 = 7\), we need to find the value of \(x\) by isolating it on one side of the equation.

First, subtract 3 from both sides to eliminate the constant term on the left side:
\[
2x + 3 - 3 = 7 - 3
\]
This simplifies to:
\[
2x = 4
\]

Next, divide bot

In [59]:
print(targetDf['Prompt'][100])

Instruction: Why is the given answer wrong under such circumstances? Provide only the final answer.
answer: Both Tom and Katie
ConstructName: Factorise a quadratic expression in the form x² - bx - c
QuestionText: Tom and Katie are arguing about factorising. Tom says \( x^{2}+5 x+6 \equiv(x+3)(x+2) \) 
Katie says \( x^{2}-5 x-6 \equiv(x-3)(x-2) \) 
Who is correct?


In [61]:
targetDf['Generated Response'][0]

'# of the two numbers and then subtracting one number from the other.\nfrom the two numbers in order.\nis the same as the difference between the two numbers in order.\nnumbers in order.\nthe smaller of the two numbers in order.\nthe sum of the two numbers in order.\nthe product of the two numbers in order.\nof the two numbers in order.\n100 times the difference of the two numbers in order.\nthe difference of the two numbers in order.\na number, the two numbers are not necessarily in the same order. the sum of the two numbers in order.\nthe square of the smaller of the two numbers in order.\nthe square of the larger of the two numbers in order.\nto the sum of the two numbers in order.\nthe sum of the two numbers in order.\nthe sum of the two numbers in order.\nthe sum of the two numbers in order.\nthe square of the smaller of the two numbers in order.\nthe sum of the two numbers in order.\nthe square of the larger of the two numbers in order.\nthe sum of the two numbers in order.\nthe s

In [62]:
targetDf['Prompt'][0]

'Instruction: Why is the given answer wrong under such circumstances? Provide only the final answer.\nanswer: ![A function machine which has 4 parts joined by arrows pointing from left to right. "y" is the first part, written on the left, followed by a horizontal arrow pointing to a rectangle that has "+ 4" written inside it, followed by a horizontal arrow pointing to a rectangle that has "square" written inside it, followed by a horizontal arrow pointing to "𝑥"]()\nConstructName: Express a non-linear equation as a function machine\nQuestionText: Which function machine matches the equation \\( y=x^{2}+4 ? \\)'